In [1]:
# imports
import glob
import json
import pandas as pd
import urllib
from bs4 import BeautifulSoup
import requests
from lxml import html
import time
import datetime

In [2]:
def scrapeProfile(profile_url):
#     print(profile_url)
    if (profile_url is None):
        currBacked = -2
        currCreated = -2
        currComments = -2
        return currBacked, currCreated, currComments
    html = urllib.request.urlopen(profile_url).read()
    soup = BeautifulSoup(html, 'lxml')
    profileDivs = soup.findAll("div", { "class" : "center" })
    currBacked = -2
    currCreated = -2
    currComments = -2
    for profileDiv in profileDivs:
#         profile does not exist
        if (profileDiv.text == "Sorry! This person is no longer active on Kickstarter."):
            currBacked = -1
            currCreated = -1
            currComments = -1
            break
#     profile still exists
    if (currBacked != -1):   
        mydivs = soup.findAll("span", { "class" : "count" })
        currBacked = mydivs[0].text[1:-1]
        currCreated = mydivs[1].text[1:-1]
        if (len(mydivs) < 3):
            currComments = 0
        else:
            currComments = mydivs[2].text[1:-1]
    return currBacked, currCreated, currComments

In [3]:
# returns the user url, given the entire JSON object as a string
def getProfileUrlFromJson(jsonObject):
    jsonObject = '{' + jsonObject.split('{', 1)[-1]
    jsonObject = jsonObject.rsplit('}', 1)[0] + '}'
    try:
        objDict = json.loads(jsonObject)
    except Exception:
        pass
    else:
        profileUrl = objDict.get('urls').get('web').get('user')
        return profileUrl

In [ ]:
# for each csv in directory
# profile_list = []
for filename in glob.glob('./Kickstarter_Data/*.csv'):
    backed_list = []
    created_list = []
    comments_list = []
    print (str(filename))
    profileCount = 0
    currFile = pd.read_csv(filename)
    profileCol = currFile[['creator']]
    for rowIdx in range(currFile.shape[0]):
        profileCount += 1
        if (profileCount%100 == 0):
            print (profileCount)
        profileJson = profileCol.iloc[[rowIdx]]
        profileJson = str(profileJson.values)
        currProfile = getProfileUrlFromJson(profileJson)
        profileBacked, profileCreated, profileComments = scrapeProfile(currProfile)
#         profile_list.append(currProfile)
        backed_list.append(profileBacked)
        created_list.append(profileCreated)
        comments_list.append(profileComments)
    ts = time.time()
    file_name_created = 'created_list_' + datetime.datetime.fromtimestamp(ts).strftime('%H-%M-%S') + '.csv'
    file_name_backed = 'backed_list_' + datetime.datetime.fromtimestamp(ts).strftime('%H-%M-%S') + '.csv'
    file_name_comments = 'comments_list_' + datetime.datetime.fromtimestamp(ts).strftime('%H-%M-%S') + '.csv'
    df_created = pd.DataFrame(created_list, columns=["created"])
    df_backed = pd.DataFrame(backed_list, columns=["backed"])
    df_comments = pd.DataFrame(comments_list, columns=["comments"])
    df_created.to_csv(file_name_created, index=False)
    df_backed.to_csv(file_name_backed, index=False)
    df_comments.to_csv(file_name_comments, index=False)